# Q-Learning with Taxi V3

### Enviorment
* Grid 5 x 5 = 25 Squares
* 5 Locations for the Passenger: R, G, B, Y or in the Taxi
* 4 Destinations: R, G, B, Y

**Discrete State Space = 25 * 5 * 4 = 500**

This is the amount of Rows of the Q-Table.

### Goal
Get the passenger & deliver to destination.

### Action Space
* 4 Directions (N, S, W, E)
* Pickup
* Drop off

**Discrete Action Space = 6**

This is the amount of Columns of the Q-Table.

### Rewards
* -1 For each move/step.
* +20 For sucessfully deliver.
* -10 For pickup out the passanger location or dropoff in other destination.

The goal is to have the minimum amount possible of negative reward, to go as fast as possible.

In [1]:
import numpy as np
import gym
import random

## Step 1: Create the Enviornment

In [2]:
env = gym.make("Taxi-v3")
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



## Step 2: Create the Q-Table

In [3]:
state_space = env.observation_space.n
action_space = env.action_space.n

Q = np.zeros((state_space, action_space))

print(Q)
print(Q.shape)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
(500, 6)


## Step 3: Define the Hyperparameters

In [4]:
total_episodes = 25000        # Total number of training episodes
total_test_episodes = 100     # Total number of test episodes
max_steps = 200               # Max steps per episode

learning_rate = 0.01          # Learning rate
gamma = 0.99                  # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.001            # Minimum exploration probability 
decay_rate = 0.01             # Exponential decay rate for exploration prob

## Step 4: Define the Epsilon-Greedy Policy
This handles the exploration/explotation trade-off.

* If a random number > Epsilon -> Explotation (Agent selects highest state-action pair value)
* Otherwise do Exploration (any random action)

In [5]:
def epsilon_greedy_policy(Q, state):
    if (random.uniform(0,1) > epsilon):
      action = np.argmax(Q[state])
    else:
      action = env.action_space.sample()
      
    return action

## Step 5: Train the Q-Learning Algorithim

In [6]:
for episode in range(total_episodes):
  # Reset the Enviornment
  state = env.reset()
  step = 0
  done = False

  # Reduce Epsilon to Reduce Exploration Probability
  epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(- decay_rate * episode)

  for step in range(max_steps):
    action = epsilon_greedy_policy(Q, state)

    # Take the Action & Observe the Rewards (r) & Outcome State (s)
    new_state, reward, done, info = env.step(action)

    # Update the Q-Table Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
    Q[state][action] = Q[state][action] + learning_rate * (reward + gamma * np.max(Q[new_state]) - Q[state][action])

    # If the game is Done we finish the episode
    if done == True:
      break

    # Set New State as State
    state = new_state

print(Q)

[[ 0.          0.          0.          0.          0.          0.        ]
 [-3.18656539 -3.15895685 -3.15839206 -3.15910522  9.62175663 -3.24189153]
 [-1.60726728 -1.59521517 -1.60017362 -1.59621394 14.11878179 -1.59308095]
 ...
 [-0.75694623 -0.68686599 -0.75685572 -0.76227654 -0.79186636 -0.87020856]
 [-2.16479778 -2.16477384 -2.16619499 -2.16600259 -2.19739293 -2.17397003]
 [-0.02989801 -0.03029391 -0.02989801  0.29793415 -0.297109   -0.1       ]]


## Step 6: Test

In [7]:
rewards = []

for episode in range(total_test_episodes):
  # Reset the Enviornment
  state = env.reset()
  step = 0
  done = False
  total_rewards = 0

  print("******************** EPISODE ", episode, " ********************")

  for step in range(max_steps):
    env.render()

    # Take the action that has the Maximum Expected Future Reward for that State
    action = np.argmax(Q[state][:])
    new_state, reward, done, info = env.step(action)
    total_rewards += reward

    if done:
      rewards.append(total_rewards)
      break
    
    state = new_state

env.close()

print('Score:', str(sum(rewards)/total_test_episodes))

Streaming output truncated to the last 5000 lines.
| : | : : |
| : : :_: |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : :_|
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : :_|
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
******************** EPISODE  51  ********************
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
 